# Vejman.dk
### Data preprocessing for accident data from vejman.dk
*Written by - Rasmus Bergman rbvp20@student.aau.dk*

This is a notebook for preprocessing the data from vejman.dk

It takes excel files from vejman.dk and turns them into a geoJSON file

### **Prerequisits**
- Downloading data from vejman.dk

### **Download Data from vejman.dk**
This is done at [Vejman.dk](https://vejman.vd.dk/query/default.do) in the tab *Avanceret søgning*.

Then navigate to the following folder:
- Vejparametre
    - Trafiksikkerhed
        - Uheldsdata
            - Uheld

There you will find all parameters which are available.

The data downloaded should contain **at least** the following parameters:
- X_Koordinat,
- Y_Koordinat,
- Adm_Vej1_Id,
- Adm_Vej2_Id

Any additional parameters can be added to the query below when downloading data (where you specify the year XXXX):
```
VÆLG
  X_Koordinat,
  Y_Koordinat,
  Adm_Vej1_Id,
  Adm_Vej2_Id
HVOR
  År = XXXX
```

It should be downloaded in the Microsoft Excel format (the other ones are very slow to download).

In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
import os
import constants as c

In [6]:
# Helper function for loading and preprocessing the vejman data
def create_dataframe(path: str, year: int) -> pd.DataFrame:
    # Sould contain the columns "ADM_VEJ1_ID", "ADM_VEJ2_ID", "X_KOORDINAT", "Y_KOORDINAT"
    df = pd.read_excel(path, header=2)

    # The parts before the "-" in the ADM_VEJ1_ID and ADM_VEJ2_ID are the official road ids
    # Thse can be used to identify roads in any dataset from Vejdirektoratet.
    # They can be looked up at cvf.dk
    # In this case they are used to identify roads in the Mastra dataset.
    df["VEJ1"] = df.ADM_VEJ1_ID.str.split("-", expand=True)[0]
    df["VEJ2"] = df.ADM_VEJ2_ID.str.split("-", expand=True)[0]
    df.drop(columns=["ADM_VEJ1_ID", "ADM_VEJ2_ID"], inplace=True)
    # The road ids are strings, but the strings are just numbers, so we convert them to integers
    df["VEJ1"] = df["VEJ1"].fillna(-1).astype(int)
    df["VEJ2"] = df["VEJ2"].fillna(-1).astype(int)

    # Sort the road ids so that the smaller id is always in the first column
    # This makes finding accidents in the same intersection easier
    df["VEJ1"], df["VEJ2"] = df[["VEJ1", "VEJ2"]].min(axis=1), df[["VEJ1", "VEJ2"]].max(axis=1)
    
    df["ÅR"] = year
    return df

In [7]:
# Load the vejman.dk data but use your own paths, these are just mine.
data = pd.concat([create_dataframe(os.path.join(c.VEJMAN_DIR, file), year) for year, file in c.VEJMAN_FILES.items()])
data.reset_index(drop=True, inplace=True)

c:\Users\rasmu\Documents\projects\P4\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\rasmu\Documents\projects\P4\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\rasmu\Documents\projects\P4\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\rasmu\Documents\projects\P4\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.X_KOORDINAT, data.Y_KOORDINAT), crs="EPSG:25832")

In [10]:
gdf.to_file(c.VEJMAN_PATH, driver="GeoJSON")